In [2]:
# Topsis on Text Summarization Model

In [3]:
!pip install transformers datasets evaluate accelerate

In [4]:
# from datasets import load_dataset

# dataset = load_dataset("ccdv/pubmed-summarization")

from datasets import load_dataset

# Load the SAMSum dataset
dataset = load_dataset("samsum")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [6]:
dataset["train"][2]

{'id': '13681000',
 'dialogue': "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style",
 'summary': 'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'}

In [7]:
train_data = dataset["train"]

In [8]:
train_data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [9]:
import pandas as pd

In [10]:
# Convert to Pandas DataFrame
train_df = pd.DataFrame(train_data)

In [11]:
train_df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [12]:
train_df.describe()

,id,dialogue,summary
count,14732,14732,14732
unique,14732,14265,14730
top,13818513,"Jimmy: Hey, guess what? My car's completely ka...",Seth's pet Oreo that he got when he was 10 is ...
freq,1,4,2


In [16]:
# # Assuming your original DataFrame is named train_df
# # Create separate DataFrames based on the length of the articles and abstracts

# # Example: Filtering based on the length of articles
# short_articles_df = train_df[train_df['article'].str.len() < 500].head(1500)
# medium_articles_df = train_df[(train_df['article'].str.len() >= 500) & (train_df['article'].str.len() < 1000)].head(1500)
# long_articles_df = train_df[train_df['article'].str.len() >= 1000].head(1500)

# # Example: Filtering based on the length of abstracts
# short_abstracts_df = train_df[train_df['abstract'].str.len() < 100].head(1500)
# medium_abstracts_df = train_df[(train_df['abstract'].str.len() >= 100) & (train_df['abstract'].str.len() < 200)].head(1500)
# long_abstracts_df = train_df[train_df['abstract'].str.len() >= 200].head(1500)

# # Concatenate the DataFrames to create new balanced DataFrames for both articles and abstracts
# balanced_articles_df = pd.concat([short_articles_df, medium_articles_df, long_articles_df])
# balanced_abstracts_df = pd.concat([short_abstracts_df, medium_abstracts_df, long_abstracts_df])

# # Combine the article and abstract DataFrames (if needed, ensure no duplication or consider intersection)
# balanced_df = pd.concat([balanced_articles_df, balanced_abstracts_df]).drop_duplicates().sample(frac=1).reset_index(drop=True)

# # Shuffle the new DataFrame to randomize the order of rows
# balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)




import pandas as pd

# Assuming train_df is already defined
# Example: train_df = pd.read_csv("your_train_data.csv")

# Example: Filtering based on the length of dialogues
short_dialogues_df = train_df[train_df['dialogue'].apply(lambda x: len(x) < 200)].iloc[:1500]
medium_dialogues_df = train_df[train_df['dialogue'].apply(lambda x: len(x) >= 200 and len(x) < 400)].iloc[:1500]
long_dialogues_df = train_df[train_df['dialogue'].apply(lambda x: len(x) >= 400)].iloc[:1500]

# Example: Filtering based on the length of summaries
short_summaries_df = train_df[train_df['summary'].apply(lambda x: len(x) < 50)].iloc[:1500]
medium_summaries_df = train_df[train_df['summary'].apply(lambda x: len(x) >= 50 and len(x) < 100)].iloc[:1500]
long_summaries_df = train_df[train_df['summary'].apply(lambda x: len(x) >= 100)].iloc[:1500]

# Concatenate the DataFrames to create new balanced DataFrames for both dialogues and summaries
balanced_dialogues_df = pd.concat([short_dialogues_df, medium_dialogues_df, long_dialogues_df], ignore_index=True)
balanced_summaries_df = pd.concat([short_summaries_df, medium_summaries_df, long_summaries_df], ignore_index=True)

# Shuffle the new DataFrames to randomize the order of rows
balanced_dialogues_df = balanced_dialogues_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_summaries_df = balanced_summaries_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Combine the dialogues and summaries DataFrames (if needed, ensure no duplication or consider intersection)
balanced_df = pd.concat([balanced_dialogues_df, balanced_summaries_df], axis=1)

# Shuffle the new DataFrame to randomize the order of rows
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [20]:
# import string
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# # Ensure required NLTK resources are downloaded
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# def preprocess_text_for_summarization(text):
#     # Convert to lowercase
#     text = text.lower()

#     # Tokenize into sentences
#     sentences = sent_tokenize(text)

#     # Process each sentence
#     processed_sentences = []
#     for sentence in sentences:
#         # Remove punctuation within the sentence
#         sentence = sentence.translate(str.maketrans('', '', string.punctuation))

#         # Tokenization
#         tokens = word_tokenize(sentence)

#         # Optional: Remove stopwords (consider keeping them for summarization to maintain meaning)
#         stop_words = set(stopwords.words('english'))
#         tokens = [token for token in tokens if token not in stop_words]

#         # Lemmatization
#         lemmatizer = WordNetLemmatizer()
#         tokens = [lemmatizer.lemmatize(token) for token in tokens]

#         # Rejoin tokens into processed sentence
#         processed_sentence = ' '.join(tokens)
#         processed_sentences.append(processed_sentence)

#     # Join processed sentences back into a single text
#     preprocessed_text = ' '.join(processed_sentences)

#     return preprocessed_text




def preprocess_text_for_summarization(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()

        # Tokenize into sentences
        sentences = sent_tokenize(text)

        # Process each sentence
        processed_sentences = []
        for sentence in sentences:
            # Remove punctuation within the sentence
            sentence = sentence.translate(str.maketrans('', '', string.punctuation))

            # Tokenization
            tokens = word_tokenize(sentence)

            # Optional: Remove stopwords (consider keeping them for summarization to maintain meaning)
            stop_words = set(stopwords.words('english'))
            tokens = [token for token in tokens if token not in stop_words]

            # Lemmatization
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(token) for token in tokens]

            # Rejoin tokens into processed sentence
            processed_sentence = ' '.join(tokens)
            processed_sentences.append(processed_sentence)

        # Join processed sentences back into a single text
        preprocessed_text = ' '.join(processed_sentences)

        return preprocessed_text
    elif isinstance(text, pd.Series):
        return text.apply(preprocess_text_for_summarization)
    else:
        raise ValueError("Unsupported input type. Only str or pd.Series are supported.")


In [21]:
from tqdm import tqdm
import pandas as pd

# Sample 5% of the rows from train_df
train_df_sampled = balanced_df.sample(frac=0.05, random_state=42)

# Apply preprocessing to the sampled data with a progress bar
tqdm.pandas(desc="Processing dialogue")
train_df_sampled['dialogue'] = train_df_sampled['dialogue'].progress_apply(preprocess_text_for_summarization)

tqdm.pandas(desc="Processing summary")
train_df_sampled['summary'] = train_df_sampled['summary'].progress_apply(preprocess_text_for_summarization)

Processing summary: 100%|██████████| 2/2 [00:00<00:00,  5.20it/s]


In [22]:
train_df_sampled.head()

,id,dialogue,summary,id,dialogue,summary
2323,13730154,eric hi free time regina u ask eric curiosity ...,regina like read book watch tv series embroide...,13812649-1,galilea go meet cortez go movie galilea movie ...,galilea cortez watch movie togehther galilea f...
3686,13727578,alex hey going greece summer saki ye grandma s...,saki alex go greece summer grandma sick alex s...,13814151-1,harry tommy join u mike dinner cinema harry we...,mike doesnt mind tommy joining dinner film har...
2974,13811305,kate talked bos today tom kate changed working...,kate talked bos today changed working hour 111...,13716850,ani well ani there something need tell guy dia...,ani company financial problem he wondering sta...
2992,13865452,pedro got promoted samantha great well deserve...,pedro got promoted,13680974,taylor wash car please dad okay taylor thanks,dad wash taylor car
468,13729874,jordan whats ernest work work work jordan free...,jordan invite ernest watch match tomorrow 8 en...,13680906,robbie hey robbie tonight trishia nothing robb...,robbie invited trishia cinema accepted trishia...


In [23]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(train_df_sampled, test_size=0.2, random_state=42)

# Define the models to be evaluated
model_names = ["bert-base-uncased", "distilbert-base-uncased", "roberta-base", "albert-base-v2", "xlnet-base-cased"]

In [24]:
!pip install rouge_score
!pip install sacrebleu
!pip install bert-score
# !pip install rouge_score

In [25]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric

# Initialize the tokenizer and model for a typical sequence-to-sequence task like summarization
tokenizer = AutoTokenizer.from_pretrained('t5-base')  # Example using T5, adapt as necessary
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

# Load summarization-specific metrics
rouge = load_metric('rouge')
bleu = load_metric('sacrebleu')
meteor = load_metric('meteor')
bertscore = load_metric('bertscore')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<ipython-input-25-caa67c8802b9>:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load th

In [31]:
# Dictionary to store results
results_dict = {}

In [21]:
# import warnings
# warnings.filterwarnings("ignore")
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from datasets import load_metric

# # Initialize metrics
# rouge = load_metric('rouge')
# bleu = load_metric('sacrebleu')
# meteor = load_metric('meteor')
# bertscore = load_metric('bertscore')

# # List of models you are evaluating
# model_names = ["t5-base", "bart-large", "pegasus-large"]

# # Dictionary to store results
# results_dict = {}

# for model_name in model_names:
#     # Load tokenizer and model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#     # Example data preparation
#     train_articles = train_df['article'].tolist()
#     train_summaries = train_df['abstract'].tolist()  # Assuming your DataFrame has 'article' and 'abstract' columns

#     # Use tqdm for a progress bar
#     for i in tqdm(range(len(train_articles)), desc=f"Processing {model_name}"):
#         article = train_articles[i]
#         reference_summary = [train_summaries[i]]

#         # Prepare the data
#         tokenized_article = tokenizer(article, return_tensors="pt", padding=True, truncation=True)

#         # Generate summary
#         with torch.no_grad():
#             summary_ids = model.generate(tokenized_article['input_ids'], num_beams=4, max_length=50, early_stopping=True)

#         generated_summary = [tokenizer.decode(summary_ids[0], skip_special_tokens=True)]

#         # Compute metrics
#         rouge_result = rouge.compute(predictions=[generated_summary], references=[reference_summary])
#         bleu_result = bleu.compute(predictions=[generated_summary], references=[reference_summary])
#         meteor_result = meteor.compute(predictions=[generated_summary], references=[reference_summary])
#         bertscore_result = bertscore.compute(predictions=[generated_summary], references=[reference_summary], lang='en')

#         # Store results
#         results_dict.setdefault(model_name, []).append({
#             "ROUGE": rouge_result['rouge1'].mid.fmeasure,  # Example: storing ROUGE-1
#             "BLEU": bleu_result['score'],
#             "METEOR": meteor_result['meteor'],
#             "BERTScore": bertscore_result['precision'][0]  # Example: storing precision from BERTScore
#         })

#     # Print average scores for each model after processing all data
#     avg_rouge = sum(d['ROUGE'] for d in results_dict[model_name]) / len(results_dict[model_name])
#     avg_bleu = sum(d['BLEU'] for d in results_dict[model_name]) / len(results_dict[model_name])
#     avg_meteor = sum(d['METEOR'] for d in results_dict[model_name]) / len(results_dict[model_name])
#     avg_bertscore = sum(d['BERTScore'] for d in results_dict[model_name]) / len(results_dict[model_name])
#     print(f"Model: {model_name}")
#     print(f"Average ROUGE: {avg_rouge:.4f}")
#     print(f"Average BLEU: {avg_bleu:.4f}")
#     print(f"Average METEOR: {avg_meteor:.4f}")
#     print(f"Average BERTScore: {avg_bertscore:.4f}")
#     print("=" * 50)

In [28]:
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric
from tqdm import tqdm
import pandas as pd

# Sample 5% of the rows from train_df
train_df_sampled = balanced_df.sample(frac=0.05, random_state=42)

# Initialize metrics
rouge = load_metric('rouge')
bleu = load_metric('sacrebleu')
meteor = load_metric('meteor')
bertscore = load_metric('bertscore')

# List of models you are evaluating
model_names = ["t5-base", "bart-large", "pegasus-large"]

# Dictionary to store results
results_dict = {}

for model_name in model_names:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Example data preparation
    train_articles = train_df_sampled['dialogue'].values.tolist()
    train_summaries = train_df_sampled['summary'].values.tolist()

    # Batch processing loop
    batch_size = 8  # Choose an appropriate batch size
    for i in tqdm(range(0, len(train_articles), batch_size), desc=f"Processing {model_name}"):
        batch_articles = train_articles[i:i+batch_size]
        batch_summaries = train_summaries[i:i+batch_size]

        # Prepare the data
        tokenized_articles = tokenizer(batch_articles, return_tensors="pt", padding=True, truncation=True)
        reference_summaries = [[train_summaries[j]] for j in range(i, min(i+batch_size, len(train_articles)))]

        # Generate summaries
        with torch.no_grad():
            summary_ids = model.generate(tokenized_articles['input_ids'], num_beams=4, max_length=50, early_stopping=True)
        generated_summaries = [tokenizer.decode(summary_ids[j], skip_special_tokens=True) for j in range(len(batch_articles))]

        # Compute metrics
        rouge_results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
        bleu_results = bleu.compute(predictions=generated_summaries, references=reference_summaries)
        meteor_results = meteor.compute(predictions=generated_summaries, references=reference_summaries)
        bertscore_results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang='en')

        # Store results
        for j in range(len(batch_articles)):
            results_dict.setdefault(model_name, []).append({
                "ROUGE": rouge_results['rouge1'][j].mid.fmeasure,
                "BLEU": bleu_results['score'][j],
                "METEOR": meteor_results['meteor'][j],
                "BERTScore": bertscore_results['precision'][j]
            })

    # Print average scores for each model after processing all data
    avg_rouge = sum(d['ROUGE'] for d in results_dict[model_name]) / len(results_dict[model_name])
    avg_bleu = sum(d['BLEU'] for d in results_dict[model_name]) / len(results_dict[model_name])
    avg_meteor = sum(d['METEOR'] for d in results_dict[model_name]) / len(results_dict[model_name])
    avg_bertscore = sum(d['BERTScore'] for d in results_dict[model_name]) / len(results_dict[model_name])
    print(f"Model: {model_name}")
    print(f"Average ROUGE: {avg_rouge:.4f}")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average METEOR: {avg_meteor:.4f}")
    print(f"Average BERTScore: {avg_bertscore:.4f}")
    print("=" * 50)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Processing t5-base:   0%|          | 0/29 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing t5-base:   0%|          | 0/29 [01:16<?, ?it/s]


AttributeError: 'Score' object has no attribute 'mid'